In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from mjol.base import *
from mjol.gan import *
from mjol.tools import *
from collections import defaultdict
import numpy as np

In [3]:
o_fn = "/ccb/salz4-3/hji20/chm13/chm13v2.0_RefSeq_Liftoff_v5.2.gff3"
o_chm13 = GAn(
    file_name = o_fn,
    file_fmt = 'gff'
)
o_chm13.build_db()

In [4]:
nw_fn = "/ccb/salz4-3/hji20/chm13/T2T_annotation_v5.3/output/mapped_reformatted.gff"
nw_chm13 = GAn(
    file_name = nw_fn,
    file_fmt = 'gff'
)
nw_chm13.build_db()

In [7]:
o_gene_coords = defaultdict(list)
o_gnames = set()

n_genes = 0
for f in o_chm13.features.values():
    if f.feature_type == "gene":
        o_gene_coords[f.chr].append((f.start, f.end, f.strand))
        if 'gene' in f.attributes:
            gname = f.attributes['gene']
        elif 'gene_name' in f.attributes:
            gname = f.attributes['gene_name']
        else:
            gname = None
        if not gname:
            print("Error : no gene name")
        o_gnames.add(gname)
        n_genes += 1

print(n_genes)

61312


In [10]:
chr_bvecs = dict()

with open('/ccb/salz4-3/hji20/hg002-q100-annotation/data/chm13v2.0.fa.fai', 'r') as fh:
    for ln in fh:
        temp = ln.strip().split('\t')
        chr_bvecs[temp[0]] = (np.zeros(int(temp[1])), np.zeros(int(temp[1])))

In [11]:
for chr in o_gene_coords:
    for st, en, strand in o_gene_coords[chr]:
        bi = 0 if strand == '+' else 1
        chr_bvecs[chr][bi][st - 1:en] = 1

In [19]:
intergenic_ginfos = []
for f in nw_chm13.features.values():
    if f.feature_type == "gene":
        bi = 0 if f.strand == '+' else 1
        s = np.sum(chr_bvecs[f.chr][bi][f.start - 1:f.end])
        if s == 0:
            if 'gene' in f.attributes:
                gname = f.attributes['gene']
            elif 'gene_name' in f.attributes:
                gname = f.attributes['gene_name']
            else:
                gname = None
            if not gname:
                print("Error : no gene name")
            if gname in o_gnames:
                print(f"Warning : additional copy detected for {gname}")
            intergenic_ginfos.append(f.uid)

In [ ]:
out_fn = "/ccb/salz4-3/hji20/chm13/out/itg.gff"
with open(out_fn, 'w') as fh:
    for uid in intergenic_ginfos:
        f = nw_chm13.features[uid]
        f.children[0].feature_type = 'transcript'
        out = f.to_gff_entry(
            include_children = True
        )
        fh.write(out)